# HistoMIL Setup Notebook

This Jupyter notebook is intended to help users get started with HistoMIL by guiding them through the process of locating and organizing the required data and files. Additionally, it will cover how to specify your logger API (set wandb API key) to enable experiment tracking and visualization using Weights & Biases (wandb).

## Getting Started

Before proceeding with this notebook, please make sure that you have followed the setup instructions provided in the project's README file. This includes creating a conda environment and installing the required dependencies.

## Organise files

Before proceeding experiments, please make sure that you organise your file in two main folders. 

(1)Data folder should include all data and the preprocessed files will also be saved here. 

(2)Experiment folder should include HistoMIL folder and others (data index, pre-trained, etc.)

All of these information will be saved in machine instance.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from HistoMIL.DATA.Cohort.location import Locations
from HistoMIL.EXP.workspace.env import Machine
from HistoMIL.EXP.workspace.env import Person

from dotenv import load_dotenv
import os
import sys
import pickle

In [ ]:
#--------------------------> init machine and person
localcohort_name = "BRCA"               # name of the cohort of patients
data_dir = f'path/to/DATA/{localcohort_name}/'
exp_dir = f'path/to/EXP/'
data_locs = Locations(  root=data_dir,
                        sub_dirs={
                                "slide":f"TCGA-{localcohort_name}/", # where slides will be (downloaded) and read
                                "tissue":"Tissue/",                  # where segmented tissue will be stored as a .pkl file
                                "patch":"Patch/",                    # where cropped patches' indices will be stored as a .h5 file
                                "patch_img":"Patch_Image/",# need to add for SSL
                                "feature":"Feature/",                # where extracted feature vectors are stored as .pt files
                                })
exp_locs = Locations(  root=exp_dir,
                        sub_dirs={
                                "src":"HistoMIL/",                   # where HistoMIL is git cloned
                                "idx":"/Data/",                      # where .csv files with cohort files and task data
                                "saved_models":"/SavedModels/",         
                                "out_files":"/OutFiles/",
                                "temp":"/Temp/",
                                "user":"/User/",                     # where User's data such as name, data_dir and exp_dir are stored as .pkl file
                                     })

machine = Machine(data_locs,exp_locs)

## Specifying Person

HistoMIL has a Person class designed to store user-related information, such as a possible username and the key for the related logger. This class can be used to specify the user's information and be passed as a parameter to relevant HistoMIL functions.

If you wish to use wandb to track and visualize your experiments with HistoMIL, you will need to specify your wandb API key. This section of the notebook will guide you through the process of doing so.

In [ ]:
# Logging information

userid = f'a user id'
username = f'a username' 
api_dir = 'path/to API.env/'                     # We assume you store your API keys in a .env file
load_dotenv(dotenv_path=f"{api_dir}API.env")


user = Person(id=userid)
user.name = username
user.wandb_api_key = os.getenv("WANDB_API_KEY") # We assume your wandb API key is named as WANDB_API_KEY in the API.env file

## Saving Person and Machine Information (Optional)

You can optionally define and save the Person and machine-related information here to use across different notebooks. To do so, create a pkl file in the selected path and define your Person object and any other relevant machine-related information. 

Note: Make sure to include the saved file in your .gitignore to prevent accidentally committing sensitive information to your repository.

In [ ]:
# save as pickle
loc = exp_locs.abs_loc("user")
with open(f"/{loc}/{localcohort_name}_machine_config.pkl", 'wb') as f:
    pickle.dump([data_locs,exp_locs,machine,user], f)